In [ ]:
import pandas as pd

# ------------------------- LOAD MATCH DATA -------------------------

df = pd.read_csv("/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase1_all.csv")

print(f" Loaded {len(df)} matches")

# ------------------------- BASIC TEAM STATS -------------------------

team_stats = {}

def update_team_stats(team, goals_for, goals_against, result):
    if team not in team_stats:
        team_stats[team] = {
            "matches": 0,
            "goals_for": 0,
            "goals_against": 0,
            "wins": 0,
            "draws": 0,
            "losses": 0
        }
    team_stats[team]["matches"] += 1
    team_stats[team]["goals_for"] += goals_for
    team_stats[team]["goals_against"] += goals_against

    if result == 1:
        team_stats[team]["wins"] += 1
    elif result == 0:
        team_stats[team]["draws"] += 1
    else:
        team_stats[team]["losses"] += 1

for _, row in df.iterrows():
    update_team_stats(row["team1"], row["home_goals"], row["away_goals"], 
                      1 if row["home_goals"] > row["away_goals"] else (0 if row["home_goals"] == row["away_goals"] else -1))
    update_team_stats(row["team2"], row["away_goals"], row["home_goals"], 
                      1 if row["away_goals"] > row["home_goals"] else (0 if row["away_goals"] == row["home_goals"] else -1))

# ------------------------- TEAM STRENGTH FEATURES -------------------------

def get_team_strength_diff(home_team, away_team):
    stats1 = team_stats.get(home_team, {})
    stats2 = team_stats.get(away_team, {})

    gf1 = stats1.get("goals_for", 0)
    ga1 = stats1.get("goals_against", 0)
    gf2 = stats2.get("goals_for", 0)
    ga2 = stats2.get("goals_against", 0)

    return (gf1 - ga1) - (gf2 - ga2)

df["team_strength_diff"] = df.apply(
    lambda row: get_team_strength_diff(row["team1"], row["team2"]), axis=1
)

# ------------------------- RECENT FORM -------------------------

def get_win_rate(team):
    stats = team_stats.get(team, {})
    return stats.get("wins", 0) / max(1, stats.get("matches", 1))

df["home_team_win_rate"] = df["team1"].apply(get_win_rate)
df["away_team_win_rate"] = df["team2"].apply(get_win_rate)
df["win_rate_diff"] = df["home_team_win_rate"] - df["away_team_win_rate"]

# ------------------------- MATCHUP HISTORY -------------------------

# Initialize tracking dictionary
history = {}

# Lists to store history-based features
past_meetings = []
past_wins = []
past_draws = []
past_losses = []

for idx, row in df.iterrows():
    home = row["team1"]
    away = row["team2"]

    matchup_key = tuple(sorted([home, away]))  # order-independent key

    # Default history
    meetings = 0
    wins = 0
    draws = 0
    losses = 0

    if matchup_key in history:
        past_results = history[matchup_key]

        meetings = len(past_results)
        for res in past_results:
            if res == 1:
                wins += 1
            elif res == 0:
                draws += 1
            else:
                losses += 1

    past_meetings.append(meetings)
    past_wins.append(wins)
    past_draws.append(draws)
    past_losses.append(losses)

    # Update history AFTER storing past stats
    result = row["outcome"]  # home team perspective
    if matchup_key not in history:
        history[matchup_key] = []
    history[matchup_key].append(result)

# Add to DataFrame
df["past_meetings"] = past_meetings
df["past_wins"] = past_wins
df["past_draws"] = past_draws
df["past_losses"] = past_losses

# ------------------------- SAVE FEATURES -------------------------

output_path = "/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase2_features.csv"
df.to_csv(output_path, index=False)
print(f"\n Feature dataset saved to {output_path} with {df.shape[0]} rows and {df.shape[1]} columns!")




In [ ]:
import os
import re

ucl_folder = "/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/champions-league-master"

month_map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05',
             'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10',
             'Nov': '11', 'Dec': '12'}

def get_date(line):
    match = re.search(r'\w+\s+(\w+)/(\d{1,2})\s+(\d{4})', line)
    if match:
        month = month_map.get(match.group(1), '01')
        day = int(match.group(2))
        year = int(match.group(3))
        return f"{year}-{month}-{day:02d}"
    return None


all_matches = []

for root, dirs, files in os.walk(ucl_folder):
    for filename in files:
        if not filename.endswith(".txt"):
            continue

        season = os.path.basename(root)
        current_date = None

        filepath = os.path.join(root, filename)
        with open(filepath, encoding="utf-8") as f:
            for line in f:
                line = line.strip()

                date = get_date(line)
                if date:
                    current_date = date

                if " v " not in line:
                    continue

                line = re.sub(r'^\d{1,2}\.\d{2}\s+', '', line)

                parts = line.split(" v ")
                if len(parts) != 2:
                    continue

                team1 = re.sub(r'\(.*?\)', '', parts[0]).strip()

                team2_score = parts[1]
                score_match = re.search(r'^(.*?)\s+(\d+)-(\d+)', team2_score)

                if not score_match:
                    continue

                team2 = re.sub(r'\(.*?\)', '', score_match.group(1)).strip()
                home_goals = int(score_match.group(2))
                away_goals = int(score_match.group(3))

                all_matches.append({
                    "season": season,
                    "date": current_date,
                    "team1": team1,
                    "team2": team2,
                    "home_goals": home_goals,
                    "away_goals": away_goals
                })

df = pd.DataFrame(all_matches)

print(f" Parsed {len(df)} matches")
print(df.head())

df.to_csv("/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/champions_league_cleaned_FINAL_FINAL.csv", index=False)
print(" Saved CSV to champions_league_cleaned_FINAL_FINAL.csv")


In [ ]:
# Load both CSVs
phase2 = pd.read_csv("/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase2_features.csv")
ucl = pd.read_csv("/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/champions_league_cleaned_FINAL_FINAL.csv")

# ----- Prepare Champions League data -----
ucl['competition'] = 'uefa.cl'
ucl['round'] = 'unknown'

# Calculate outcome
ucl['outcome'] = ucl.apply(lambda r: 1 if r['home_goals'] > r['away_goals'] else (0 if r['home_goals'] == r['away_goals'] else -1), axis=1)

# Add missing feature columns (fill default values)
for col in ['team_strength_diff', 'home_team_win_rate', 'away_team_win_rate',
            'win_rate_diff', 'past_meetings', 'past_wins', 'past_draws', 'past_losses']:
    ucl[col] = 0

# Convert dates
ucl['date'] = pd.to_datetime(ucl['date'], errors='coerce')
phase2['date'] = pd.to_datetime(phase2['date'], errors='coerce')

# ----- Avoid duplicates -----
def is_duplicate(ucl_row):
    return ((phase2['date'] == ucl_row['date']) &
            (phase2['team1'] == ucl_row['team1']) &
            (phase2['team2'] == ucl_row['team2']) &
            (phase2['home_goals'] == ucl_row['home_goals']) &
            (phase2['away_goals'] == ucl_row['away_goals'])).any()

new_matches = ucl[~ucl.apply(is_duplicate, axis=1)]

print(f" New Champions League matches to add: {len(new_matches)}")

# ----- Merge -----
combined = pd.concat([phase2, new_matches], ignore_index=True)
combined = combined.sort_values(by='date').reset_index(drop=True)

# ----- Save -----
final_path = "/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase2_features_merged.csv"
combined.to_csv(final_path, index=False)

print(f" Final merged CSV saved to: {final_path}")
print(f"Total rows after merge: {len(combined)}")


In [ ]:
# LOAD MERGED CSV
df = pd.read_csv("/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase2_features_merged.csv")

# Better team name cleaner
def clean_name(name):
    if pd.isna(name):
        return ""
    
    # Convert to string and strip whitespace
    name = str(name).strip()
    
    # Remove anything in brackets like (ESP) or (FRA)
    name = re.sub(r'\s*\([A-Z]+\)\s*', '', name)
    
    # Create a lowercase version for case-insensitive matching
    name_lower = name.lower()
    
    # Comprehensive replacements dictionary
    replacements = {
        "internazionale milano": "Inter Milan",
        "atlético de madrid": "Atletico Madrid",
        "atletico de madrid": "Atletico Madrid",
        "sporting clube de braga": "Braga",
        "bayern münchen": "Bayern Munich",
        "real sociedad de fútbol": "Real Sociedad",
        "paris saint germain": "Paris Saint-Germain",
        "paris saint-germain fc": "Paris Saint-Germain",
        "sport lisboa e benfica": "Benfica",
        "feyenoord rotterdam": "Feyenoord",
        "1. fc union berlin": "Union Berlin",
        "fc københavn": "Copenhagen",
        "rb leipzig": "Leipzig",
        "ss lazio": "Lazio", 
        "ssc napoli": "Napoli",
        "psv eindhoven": "PSV",
        "real madrid cf": "Real Madrid",
        "real madrid c.f.": "Real Madrid",
        "fc salzburg": "Salzburg",
        "fc red bull salzburg": "Salzburg",
        "manchester city fc": "Manchester City",
        "leeds united fc": "Leeds United",
        "queens park rangers fc": "QPR",
        "coventry city fc": "Coventry City",
        "watford fc": "Watford",
        "portsmouth fc": "Portsmouth",
        "cardiff city fc": "Cardiff City",
        "norwich city fc": "Norwich City",
        "preston north end fc": "Preston North End",
        "derby county fc": "Derby County",
        "oxford united fc": "Oxford United"
    }
    
    # Check for exact matches first (case insensitive)
    for key, value in replacements.items():
        if key in name_lower:
            return value
    
    # If no direct replacement, then clean the name
    # Remove common suffixes/prefixes 
    name = re.sub(r'\b(FC|C\.F\.|CF|AC|SSC|RCD|AFC|KV|SV|SC|SL|FF|United|Club|Sporting|Red Bull|RB)\b', '', name, flags=re.IGNORECASE)
    
    # Collapse extra spaces and strip
    name = re.sub(r'\s+', ' ', name).strip()
    
    # Title case to normalize casing
    name = name.title()
    
    return name



# APPLY CLEANING IN PLACE
df["team1"] = df["team1"].apply(clean_name)
df["team2"] = df["team2"].apply(clean_name)


# Sort by date so historical stats can build over time
df = df.sort_values("date")

# Initialize team history & matchup history
team_stats = {}
matchup_history = {}

# NEW COLUMNS
past_meetings = []
past_wins = []
past_draws = []
past_losses = []
team_strength_diff = []
home_win_rate = []
away_win_rate = []
win_rate_diff = []

# LOOP through each match
for idx, row in df.iterrows():
    home = row["team1"]  # Changed from team1_clean
    away = row["team2"]  # Changed from team2_clean
    home_goals = row["home_goals"]
    away_goals = row["away_goals"]

    # --- Outcome ---
    if home_goals > away_goals:
        outcome = 1
    elif home_goals == away_goals:
        outcome = 0
    else:
        outcome = -1

    # --- Past matchups ---
    matchup_key = tuple(sorted([home, away]))
    meetings = matchup_history.get(matchup_key, [])

    past_meetings.append(len(meetings))
    past_wins.append(meetings.count(1))
    past_draws.append(meetings.count(0))
    past_losses.append(meetings.count(-1))

    # --- Update matchup history ---
    matchup_history.setdefault(matchup_key, []).append(outcome)

    # --- Win Rates ---
    home_record = team_stats.get(home, {"matches": 0, "wins": 0})
    away_record = team_stats.get(away, {"matches": 0, "wins": 0})

    home_wr = home_record["wins"] / home_record["matches"] if home_record["matches"] > 0 else 0
    away_wr = away_record["wins"] / away_record["matches"] if away_record["matches"] > 0 else 0

    home_win_rate.append(home_wr)
    away_win_rate.append(away_wr)
    win_rate_diff.append(home_wr - away_wr)

    # --- Strength diff ---
    team1_strength = (home_record["wins"] - (home_record["matches"] - home_record["wins"]))
    team2_strength = (away_record["wins"] - (away_record["matches"] - away_record["wins"]))
    team_strength_diff.append(team1_strength - team2_strength)

    # --- Update team_stats ---
    team_stats.setdefault(home, {"matches": 0, "wins": 0})
    team_stats.setdefault(away, {"matches": 0, "wins": 0})

    team_stats[home]["matches"] += 1
    team_stats[away]["matches"] += 1

    if outcome == 1:
        team_stats[home]["wins"] += 1
    elif outcome == -1:
        team_stats[away]["wins"] += 1

# Update dataframe
df["past_meetings"] = past_meetings
df["past_wins"] = past_wins
df["past_draws"] = past_draws
df["past_losses"] = past_losses
df["team_strength_diff"] = team_strength_diff
df["home_team_win_rate"] = home_win_rate
df["away_team_win_rate"] = away_win_rate
df["win_rate_diff"] = win_rate_diff

# Save FINAL CSV
final_path = "/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase2_features_FINAL_ALL_UPDATED.csv"
df.to_csv(final_path, index=False)
print(" FINAL CLEANED CSV SAVED:", final_path)